## Retrieve polygons

This Python notebook is responsible for associating the coordinates (X, Y) of shot attempts with the corresponding sector. This step is performed on the three datasets obtained earlier using R. 

This operation was carried out using the Shapely library, which allows for the creation of Point and Polygon objects and checks whether a point lies within a polygon using the `contains` and `touches` functions of the Polygon class. 

The script essentially recreates a CSV identical to the original, with the addition of the `sector_id` column that indicates the ID of the polygon in which the shot was taken.

In [1]:
from shapely.geometry import Point, Polygon
import pandas as pd

shot_map_name = 'AVVERSARIO_shot_map_po' # BRESCIA_shot_map_po, LBA_shot_map_po

df = pd.read_csv('../csv/polygons.csv')
df_shot_map = pd.read_csv('../csv/' + shot_map_name + '.csv')

In [2]:
# retrieve dictionary with polygons
polygons = {}

polygon_ids = set(df.shapeId)
for polygon_id in polygon_ids:
    df_temp = df[df.shapeId == polygon_id]
    points = list(zip(df_temp.pointX, df_temp.pointY))
    polygons.update({polygon_id:points})

In [3]:
def retrieve_polygon_id(row):
    """
    function to retrieve the polygon of the point
    """
    p = Point(row['xx'], row['yy'])
    for key, value in polygons.items():
        poly = Polygon(value)
        if poly.contains(p) or poly.touches(p):
            return key
    return -1

In [4]:
df_shot_map['sector_id'] = df_shot_map.apply(lambda row: retrieve_polygon_id(row), axis=1)

In [5]:
df_shot_map.groupby(['sector_id'])['sector_id'].count()

sector_id
0    244
1     21
2     62
3     45
4     23
5     40
6    113
7    114
8     22
Name: sector_id, dtype: int64

In [6]:
df_shot_map['sector_id'] = df_shot_map['sector_id'].astype(int)
df_shot_map.to_csv('../csv/' + shot_map_name + '_with_sector_id.csv')

In [7]:
df_shot_map.head()

,game_id,description,assist,player,result,periodTime,totalTime,xx,yy,sector_id
0,Milano Trento game 1,2 punti segnato,NaN,Shavon Shields,made,34,34,1020,644,3
1,Milano Trento game 1,2 punti sbagliato,NaN,Stefano Tonut,missed,102,102,825,196,0
2,Milano Trento game 1,2 punti sbagliato,NaN,Nicolò Melli,missed,102,102,825,196,0
3,Milano Trento game 1,3 punti segnato,Nicolò Melli,Shavon Shields,made,128,128,1020,896,7
4,Milano Trento game 1,3 punti segnato,NaN,Shabazz Napier,made,218,218,300,840,6
